In [1]:
# Import necessary libraries
import os
import cv2
import numpy as np
from ur3e_bc.modules import UR3EDataset
from ur3e_bc.models import UR3EBCModel
from torch.utils.data import Dataset, DataLoader

import time


/home/students/.local/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
def visualize_batch(batch, batch_idx=0):
    """
    Visualizes a batch of data from the dataloader in a single window.

    Args:
        batch (dict): A batch of data from the dataloader.
        batch_idx (int): Index within the batch to visualize (default: 0).
    """
    # Extract batch data for the selected sample
    front_cam = batch["front_cam"][batch_idx].cpu().numpy()  # (3, H, W, 3)
    side_cam = batch["side_cam"][batch_idx].cpu().numpy()  # (3, H, W, 3)
    hand_cam = batch["hand_cam"][batch_idx].cpu().numpy()  # (3, H, W, 3)

    

    # Convert images to OpenCV format (H, W, C) and scale up for visibility
    def preprocess_img(img):
        """
        Preprocess image from [f, H, W, C] to OpenCV format [H, W, C] for display.
        """
        # img = np.transpose(img, (1, 2, 0))  # Convert [f, H, W, C] -> [H, W, f, C] -> [H, W, C]
        img = np.clip(img * 255, 0, 255).astype(np.uint8)  # Convert back to 0-255 for display
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                
        return img  # Return as H, W, C

    front_cam_imgs = [preprocess_img(front_cam[i]) for i in reversed(range(3))]
    side_cam_imgs = [preprocess_img(side_cam[i]) for i in reversed(range(3))]
    hand_cam_imgs = [preprocess_img(hand_cam[i]) for i in reversed(range(3))]

    top_row = np.concatenate(front_cam_imgs, axis=1)
    middle_row = np.concatenate(side_cam_imgs, axis=1)
    bottom_row = np.concatenate(hand_cam_imgs, axis=1)

    all_images = np.vstack([top_row, middle_row, bottom_row])

    # Get image dimensions
    H, W, _ = all_images.shape
    step_x = W // 3  # Width of each column
    step_y = H // 3  # Height of each row

    # Titles for rows
    row_titles = ["Front Cam", "Side Cam", "Hand Cam"]
    col_titles = ["t", "t-1", "t-2"]  # Reordered column labels

    font_scale = 0.6  # Reduce font size
    thickness = 1  # Reduce text thickness

    # Add row titles
    for i, title in enumerate(row_titles):
        y_pos = step_y * i + step_y // 2
        cv2.putText(all_images, title, (5, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 
                    font_scale, (0, 255, 0), thickness, cv2.LINE_AA)

    # Add column titles
    for i, title in enumerate(col_titles):
        x_pos = step_x * i + step_x // 2 - 15
        cv2.putText(all_images, title, (x_pos, 20), cv2.FONT_HERSHEY_SIMPLEX, 
                    font_scale, (0, 255, 0), thickness, cv2.LINE_AA)


    # Display the images in one window
    cv2.imshow("Camera Views: Front, Side, Hand (t-2, t-1, t)", all_images)
   

    # Optionally print numerical data for the current sample
    ee_pose = batch["ee_pose"][batch_idx].cpu().numpy()  # (3,7)
    ee_velocity = batch["ee_velocity"][batch_idx].cpu().numpy()  # (6)
    hole_pose = batch["hole_pose"][batch_idx].cpu().numpy()  # (7)
    state = batch["state"][batch_idx].cpu().numpy()  # (1)
    
    print("image size: {}".format(batch["front_cam"].shape))
    print("\nEnd Effector Pose Over Time:")
    print(f"  t-2: {ee_pose[0]}")
    print(f"  t-1: {ee_pose[1]}")
    print(f"  t:   {ee_pose[2]}")
    
    print("\nEnd Effector Velocity:")
    print(f"  [vx, vy, vz, wx, wy, wz]: {ee_velocity}")
    
    print("\nHole Position and Orientation:")
    print(f"  [x, y, z, qx, qy, qz, qw]: {hole_pose}")

    # Print the current state
    state_labels = {0: "Move", 1: "Insert", 2: "Rotate"}
    print(f"\nCurrent State: {state_labels.get(state.item(), 'Unknown')} ({state.item()})")

    cv2.waitKey(0)  # Wait for key press
    # cv2.destroyAllWindows()  # Close the window after key press


In [3]:
# Define the directory for the zip files (data storage directory)
zip_dir = "/home/students/ur3e_mujoco_tasks/data/data_new"

# Define the batch size
batch_size = 16

# Instantiate the dataset
dataset = UR3EDataset(zip_dir)
print(dataset.episode_dict)

# Create a dataloader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)


{'20250410_111830_0.zip': ['20250410_111830_0_0.pkl', '20250410_111830_0_1.pkl', '20250410_111830_0_2.pkl', '20250410_111830_0_3.pkl', '20250410_111830_0_4.pkl', '20250410_111830_0_5.pkl', '20250410_111830_0_6.pkl', '20250410_111830_0_7.pkl', '20250410_111830_0_8.pkl', '20250410_111830_0_9.pkl', '20250410_111830_0_10.pkl', '20250410_111830_0_11.pkl', '20250410_111830_0_12.pkl', '20250410_111830_0_13.pkl', '20250410_111830_0_14.pkl', '20250410_111830_0_15.pkl', '20250410_111830_0_16.pkl', '20250410_111830_0_17.pkl', '20250410_111830_0_18.pkl', '20250410_111830_0_19.pkl', '20250410_111830_0_20.pkl', '20250410_111830_0_21.pkl', '20250410_111830_0_22.pkl', '20250410_111830_0_23.pkl', '20250410_111830_0_24.pkl', '20250410_111830_0_25.pkl', '20250410_111830_0_26.pkl', '20250410_111830_0_27.pkl', '20250410_111830_0_28.pkl', '20250410_111830_0_29.pkl', '20250410_111830_0_30.pkl', '20250410_111830_0_31.pkl', '20250410_111830_0_32.pkl', '20250410_111830_0_33.pkl', '20250410_111830_0_34.pkl', '20

In [ ]:
for batch in dataloader:
    for i in range(batch_size):
        visualize_batch(batch, batch_idx=i) 
        time.sleep(0.2)

cv2.destroyAllWindows()

image size: torch.Size([16, 3, 224, 224, 3])

End Effector Pose Over Time:
  t-2: [-0.19401896 -0.38869482  0.15481648  0.0060899   0.07246849 -0.66552585
  0.74282336]
  t-1: [-0.19401896 -0.38869482  0.15481648  0.0060899   0.07246849 -0.66552585
  0.74282336]
  t:   [-0.19401896 -0.38869482  0.15481648  0.0060899   0.07246849 -0.66552585
  0.74282336]

End Effector Velocity:
  [vx, vy, vz, wx, wy, wz]: [-0.01965472  0.09789786 -0.00544988  0.05441879 -0.02100503  0.34510487]

Hole Position and Orientation:
  [x, y, z, qx, qy, qz, qw]: [ 0.25044718 -0.2994604   0.06007347  0.          0.         -0.28070763
  0.9597933 ]

Current State: Move (0)
image size: torch.Size([16, 3, 224, 224, 3])

End Effector Pose Over Time:
  t-2: [-0.19401896 -0.38869482  0.15481648  0.0060899   0.07246849 -0.66552585
  0.74282336]
  t-1: [-0.19401896 -0.38869482  0.15481648  0.0060899   0.07246849 -0.66552585
  0.74282336]
  t:   [-0.18024717 -0.38995147  0.15455586  0.00633335  0.06712457 -0.63949984
 

In [ ]:
model = UR3EBCModel()

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [ ]:
batch = next(iter(dataloader))
a,b,c = model.forward(batch['front_cam'],batch['side_cam'],batch['hand_cam'],batch['ee_pose'],batch['joints'])

print(a)
print(b)
print(c)

tensor([[0.2880, 0.0639, 0.2980, 0.1088, 0.0642, 0.0310],
        [0.2903, 0.0727, 0.3001, 0.1155, 0.0782, 0.0290],
        [0.2902, 0.0658, 0.2990, 0.1194, 0.0707, 0.0286],
        [0.2808, 0.0598, 0.3033, 0.1254, 0.0624, 0.0289],
        [0.2841, 0.0610, 0.3016, 0.1198, 0.0680, 0.0331],
        [0.2799, 0.0653, 0.3071, 0.1259, 0.0717, 0.0349],
        [0.2798, 0.0649, 0.3089, 0.1255, 0.0660, 0.0324],
        [0.2826, 0.0665, 0.3066, 0.1247, 0.0709, 0.0303],
        [0.2827, 0.0603, 0.2994, 0.1322, 0.0633, 0.0382],
        [0.2799, 0.0668, 0.3042, 0.1233, 0.0674, 0.0383],
        [0.2791, 0.0591, 0.3098, 0.1457, 0.0683, 0.0465],
        [0.2788, 0.0584, 0.3045, 0.1314, 0.0673, 0.0376],
        [0.2780, 0.0645, 0.3123, 0.1313, 0.0668, 0.0349],
        [0.2723, 0.0686, 0.3207, 0.1363, 0.0673, 0.0375],
        [0.2791, 0.0608, 0.3115, 0.1382, 0.0661, 0.0405],
        [0.2807, 0.0623, 0.3089, 0.1428, 0.0630, 0.0461]],
       grad_fn=<AddmmBackward0>)
tensor([[-0.0880,  0.0960,  0.1876, -0